# Introduction to EOFs

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Note</p>
    This content is under construction.
</div>

---

## Overview

In this notebook, we will provide some background on empirical orthogonal functions (EOFs), develop some intuition, and cover the relevant mathematics.

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| Linear algebra | Necessary | |

- **Time to learn**: 20 minutes

---

## Imports

In [ ]:
import numpy

## Introduction

EOFs are commonly used to study the spatial and temporal variability of geophysical fields like sea surface temperature, sea level pressure, or geopotential height. Similar to how Fourier analysis allows us to decompose a time series into a sum of sinusoids of different frequencies, EOF analysis allows us to decompose a geophysical field into a set of mutually orthogonal spatial patterns and corresponding time series that are determined by the data (so, empirically). Terminology varies across texts, but here we will refer to the spatial patterns as "EOFs" and the time series as the "principal components" (PCs). 

Each EOF-PC pair can be thought of as a *mode of variability* in the data, but it is not necessary that these modes represent something physical. Each mode comes with a corresponding variance fraction that tells us how important the mode is. This is often phrased something like "the first mode explains 25% of the variance in the data," and we will discuss what this means mathematically in the following sections. If a mode explains a large fraction of the total variance, it is more likely to be capturing some *physical* mode of variability, like the [El Niño Southern Oscillation (ENSO)](https://www.ncei.noaa.gov/access/monitoring/enso/) or [North Atlantic Oscillation (NAO)](https://www.ncei.noaa.gov/access/monitoring/nao/).

## The method

The steps required to compute EOFs and PCs are as follows:
1. Organize your data into a 2-dimensional matrix $\mathbf F$ and remove the mean from each time series
1. Calculate the covariance matrix $\mathbf R=\mathbf F\mathbf F^\mathrm T$
1. Find the eigenvectors (EOFs) $\mathbf e_m\in\mathbf E$ and eigenvalues $\lambda_m\in\mathbf\Lambda$ of $\mathbf R$ by solving $\mathbf R\mathbf E=\mathbf\Lambda\mathbf E$
1. Order the eigenectors by their eigenvalues, since the eigenvalues are proportional to the fraction of variance explained by that mode
1. Calculate the PCs for each mode with $\mathbf c_m=\mathbf e_m^\mathrm T\cdot\mathbf F$ (i.e. projecting your data onto each eigenvector/EOF)
1. Find the variance fraction for each mode with $\lambda_m\Large/\normalsize\sum_i^M\lambda_i$

## Deriving the eigenvalue problem

It is not obvious why solving the eigenvalue problem for the covariance matrix results in spatial patterns that align well with the data. The following explanation follows those given by Preisendorfer (1988), Peixoto et al. (1992), and Björnsson and Venegas (1997).

Let $f(x, t)$ be a geophysical field recorded over locations $x_1, x_2, \dots, x_M$ and times $t_1, t_2, \dots, t_N$. In matrix notation, we can write each map of $M$ locations as the column vector $\mathbf{f}_n=\begin{bmatrix}f_{1n} & f_{2n} & \cdots & f_{Mn}\end{bmatrix}^\mathrm T$, where $n=1, \dots, N$. We then have an $M\times N$ matrix that can be written as $\mathbf F=\begin{bmatrix}\mathbf{f}_1 & \mathbf{f}_2 & \cdots & \mathbf{f}_N\end{bmatrix}$. 

Now, imagine that we only have three locations to collect data. Then each $\mathbf{f}_n$ would be a 3-dimensional vector, and the whole set could be represented as a linear combination of three arbitrary basis vectors. This generalizes such that the vector space $\mathbf F$ is spanned by an arbirary unit basis $\{\mathbf u_1,\mathbf u_2,\dots,\mathbf u_M\}$. It is very likely that some of the vectors $\mathbf f_n$ are correlated as a result of some physical process. The goal of the EOF analysis is then to find an orthogonal unit basis $\{\mathbf e_1,\mathbf e_2,\dots,\mathbf e_M\}$ that aligns well with these vector "clusters". This is now an optimization problem. To maximize the alignment between the basis and observations, we will maximize the projection of the vectors $\mathbf f_n$ onto each basis vector using the sum of squares:
\begin{equation}
\sum_{n=1}^N(\mathbf f_n \cdot \mathbf e_m)^2
\end{equation}
for $m=1, 2, \dots, M$. We also require mutual orthonormality for the basis: $\mathbf e_i\cdot\mathbf e_j=\delta_{ij}$. Now, let's define this quantity as $\psi(\mathbf e_m)$:
\begin{equation}
\begin{aligned}
\psi(\mathbf e_m)\equiv\sum_{n=1}^N(\mathbf f_n \cdot \mathbf e_m)^2
&=\sum_{n=1}^N(\mathbf e_m^\mathrm T \mathbf f_n)(\mathbf f_n^\mathrm T\mathbf e_m)\\
&=\mathbf e_m^\mathrm T \sum_{n=1}^N(\mathbf f_n\mathbf f_n^\mathrm T\mathbf) e_m\\
&=\mathbf e_m^\mathrm T \mathbf F\mathbf F^\mathrm T\mathbf e_m\\
&=\mathbf e_m^\mathrm T \mathbf R\mathbf e_m
\end{aligned}
\end{equation}
where $\mathbf R=\mathbf F\mathbf F^\mathrm T$ is the covariance matrix of $\mathbf F$.

We now seek the extrema of $\psi(\mathbf e)$ (dropping the subscript $m$ for convenience), which satisfy $\psi(\mathbf e+\delta\mathbf e)=\psi(\mathbf e)$ for a small change $\delta\mathbf e$ in the direction of $\mathbf e$. Then
\begin{equation}
\begin{aligned}
\psi(\mathbf e+\delta\mathbf e)&=(\mathbf e+\delta\mathbf e)^\mathrm T\mathbf R(\mathbf e+\delta\mathbf e)\\
&=\mathbf e^\mathrm T\mathbf R\mathbf e+2(\delta\mathbf e)^\mathrm T\mathbf R\mathbf e + (\delta\mathbf e)^\mathrm T\mathbf R\delta\mathbf e\\
&\approx\mathbf e^\mathrm T\mathbf R\mathbf e+2(\delta\mathbf e)^\mathrm T\mathbf R\mathbf e\\
&=\psi(\mathbf e)+2(\delta\mathbf e)^\mathrm T\mathbf R\mathbf e
\end{aligned}
\end{equation}
where we only keep the terms first order in $\delta\mathbf e$. The optimization condition implies
\begin{equation}
(\delta\mathbf e)^\mathrm T\mathbf R\mathbf e=0
\end{equation}
By the orthonormality of the basis, we also require that these variations only change the direction of $\mathbf e$, so $(\mathbf e+\delta\mathbf e)^\mathrm T(\mathbf e+\delta\mathbf e)=1\Rightarrow(\delta\mathbf e)^\mathrm T\mathbf e=0$, again only keeping terms first order in $\delta\mathbf e$. We can combine these two conditions, provided that we multiply the orthonormality condition by a constant that has the same units as the entries of $\mathbf R$:
\begin{equation}
(\delta\mathbf e)^\mathrm T\mathbf R\mathbf e-\lambda(\delta\mathbf e)^\mathrm T\mathbf e=(\delta\mathbf e)^\mathrm T[\mathbf R\mathbf e-\lambda\mathbf e]=0
\end{equation}
Since $\delta\mathbf e$ is arbitrary, we must have $\mathbf R\mathbf e-\lambda\mathbf e=0$, the eigenvalue problem for $\mathbf R$, or for nontrivial solutions, $|\mathbf R-\lambda\mathbf I|=0$.

---

## Summary
In this notebook, we covered some background on EOFs, the steps required to carry out an EOF analysis, and some of the math behind the analysis.

### What's next?
An example of carrying out an EOF analysis on idealized data using NumPy.

## Resources and references

Björnsson, H., & Venegas, S. A. (1997). A manual for EOF and SVD analyses of climatic data. *CCGCR Report*, *97*(1), 112-134.

Peixoto, J. P., Oort, A. H., & Lorenz, E. N. (1992). *Physics of climate* (Vol. 520). New York: American Institute of Physics.

Preisendorfer, R. Ä. N. Ü. (1988). Principal component analysis in meteorology and oceanography. *Elsevier Sci. Publ.*, *17*, 425.